In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Data Overview

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

In [4]:
%%sql
SELECT MIN(population), MAX(population),
       MIN(population_growth), MAX(population_growth)
  FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


# Exploring Outliers

Countries with the minimum population:

In [5]:
%%sql
SELECT name AS Country, population
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts
                    );

Done.


Country,population
Antarctica,0


Countries with the maximum population:

In [6]:
%%sql
SELECT name AS Country, population
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                    );

Done.


Country,population
World,7256490011


# Exploring Average Population and Area

## Summary statistics recomputed

In [7]:
%%sql
SELECT MIN(population), MAX(population),
       MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


## Average Population and Area

In [8]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


# Finding Densely Populated Countries

Countries that have the population above average and area below average.

In [9]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
       AND area < (SELECT AVG(area)
                     FROM facts
                    WHERE name <> 'World'
                  );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


# Exploring Top-listed Countries

## Top 5 crowded countries

In [15]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World' AND name <> 'European Union'
 ORDER BY population DESC
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14


## Top 5 countries with more water than land

In [16]:
%%sql
SELECT *
  FROM facts
 ORDER BY area_water / area_land DESC
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92


## Top 5 countries with the most people added next year

In [18]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World'
 ORDER BY population * population_growth DESC
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22


## Top 10 countries with highest population/area ratios

In [21]:
%%sql
SELECT *
  FROM facts
 ORDER BY population / area DESC
 LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88
